# 二. k近邻算法

k近邻法(`k-nearest neighbor, k-NN`)是一种基本分类和回归方法(`Cover & Hart, 1968`)。k近邻法的输入为实例的特征向量，对应于特征空间的点，输出为实例的类别，可以取多类。分类时，对新的实例，根据其k个最近邻的训练实例的类别，根据其k个最近邻的训练实例的类别，通过多数表决等方式进行预测。

k近邻法不具显式的学习过程，k值的选择、距离度量以及分类决策规则是k近邻法的三要素。难点在于如何高效地定位到输入实例的k个最近邻居。

## 1. k近邻算法

**算法3.1（k近邻法）**
- 输入：训练数据集
$$
T=\{(x_1, y_1), (x_2, y_2), ..., (x_N, y_N)\}
$$
其中，$x_i\in\mathbf{x}\subset \mathbf{R^n}$为实例的特征向量，$y_i\in \mathbf{y}=\{c_1,c_2,...,c_K\}$为实例的类别，$i=1,2,...,N$；实例特征向量$x$；
- 输出：实例$x$的所属类$y$
- 算法过程
    - 根据给定的距离度量，在训练集$T$中找出与$x$最近邻的$k$个点，涵盖这$k$个点的$x$的近邻记作$N_K(x)$
    - 在$N_k(x)$中根据分类决策规则决定$x$类别$y$
    $$
    y=\mathrm{arg} \max_{c_j} \sum_{x_i\in N_k(x)} I(y_i=c_i),i=1,2,...,N;j=1,2,...,K
    $$
    上式中，I为指示函数，即当$y_i=c_j$时$I$为1，否则$I$为0.

## 2. 距离度量

- $L_p$距离

设特征空间$\mathbf{X}$是n维实数向量空间$\mathbf{R^n}$，$x_i, x_j\in \mathbf{x}, x_i = (x^{(1)}_i,x^{(2)}_i,...,x^{(n)}_i)^T,x_j=(x_j^{(1)}, x_j^{(2)},... ,x_j^{(n)})^T,x_i,x_j$的$L_p$距离定义为
$$
L_p(x_i,x_j)=\left(\sum_{l=1}^n|x_i^{(l)}-x_j^{(l)}|^p\right)^{\frac{1}{p}}
$$
这里$p\ge 1$。

当$p=2$时，称为欧式距离(`Euclidean distance`)，即
$$
L_2(x_i,x_j)=\left(\sum_{l=1}^n|x_i^{(l)}-x_j^{(l)}|^2\right)^{\frac{1}{2}}
$$

当$p=1$时，称为曼哈顿距离(`Manhattan distance`)，即
$$
L_1(x_i,x_j)=\left(\sum_{l=1}^n|x_i^{(l)}-x_j^{(l)}|\right)
$$

当$p=\infty$时，它是各个坐标距离的最大值，即
$$
L_{\infty}(x_i,x_j)=\max_l|x_i^{(l)}-x_j^{(l)}|
$$

In [14]:
import numpy as np
import pandas as pd
import networkx as nx

In [85]:
def distance(xi, xj, p=2):
    return np.sum((np.abs(xi - xj))**p)**(1/p)

In [86]:
x1 = np.array([1,2,3])
x2 = np.array([3,2,1])

In [87]:
distance(x1, x2, p=1)

4.0

## 3. `kd`树

kd树是一种对k维空间中的实例点进行存储以便对其进行快速检索的树型数据结构。kd树是二叉树，表示对k维空间的一个划分。构造kd树相当于不断地用垂直于坐标轴的超平面将k维空间切分，构成一系列的k维超矩形区域。

kd树的每一个结点对应于一个k维矩形区域。

**算法3.2（构造平衡kd数）**
- 输入：k维空间数据集$T=\{(x_1, y_1), (x_2, y_2), ..., (x_N, y_N)\}$，其中$x_i = (x^{(1)}_i,x^{(2)}_i,...,x^{(n)}_i)^T,i=1,2,...,N$
- 输出：kd平衡树
- 算法过程：
    - 开始：构造根结点，根结点对应于包含T的k维空间的超矩形区域。选择以$x^{(1)}$为坐标轴，以T中所有实例的$x^{(1)}$坐标的*中位数*为切分点，将根结点对应的超矩形区域切分成两个子区域。切分由通过切分点并与坐标轴$x^{(1)}$垂直的超平面实现。由根结点生成深度为1的左右子结点：左子区域对应$x^{(1)}$小于切分点的子区域，右子区域对应$x^{(1)}$大于切分点的子区域。将落在切分超平面上的实例点保存在根结点。
    - 重复：对深度为j的结点，选择$x^{(l)}$为切分的坐标轴，$l=(j \text{mod} k) + 1$，以该节点的区域所有实例的$x^{(1)}$坐标的中位数为切分点，将该结点对应的超矩形区域切分为两个子区域。切分由通过切分点并与坐标轴$x^{(j)}$垂直的超平面实现。由根结点生成深度为$j+1$的左右子结点：左子区域对应$x^{(l)}$小于切分点的子区域，右子区域对应$x^{(l)}$大于切分点的子区域。将落在切分超平面上的实例点保存在该结点。
    - 直到两个区域没有实例存在时停止。从而形成kd树的区域划分。
 

> - 非叶结点上可能没有数据（中位数）
> - 数据都保留结点上

In [69]:
def generate_kd_tree(df):
    X = df[df.columns[:-1]]  # 输入实例
    y = df[df.columns[-1]]  # 类别
    k = len(X.columns)  # X的维度k
    X_columns = X.columns
    kd_tree = nx.DiGraph()
    node_id = 0
    no_tag_nodes = [node_id]
    kd_tree.add_node(node_id)
    kd_tree.nodes[node_id]["X"] = X
    kd_tree.nodes[node_id]["y"] = y
    i = 0
    while no_tag_nodes:
        new_nodes = []
        dim = i % k  # 当前的维度
        for node in no_tag_nodes:
            c_X = kd_tree.nodes[node]["X"]
            c_y = kd_tree.nodes[node]["y"]
            x_dim = c_X[X_columns[dim]]
            if len(np.unique(x_dim)) >= 2:  # 如果有2个以上样本，则继续分
                median = np.median(c_X[X_columns[dim]].values)
                kd_tree.nodes[node]["dim"] = dim  # 结点的切分维度
                kd_tree.nodes[node]["median"] = median
                l_indices = c_X[X_columns[dim]] < median  # 左子区域
                m_indices = c_X[X_columns[dim]] == median  # 留在node中
                r_indices = c_X[X_columns[dim]] > median  # 右子区域
                l_X, l_y = c_X[l_indices], c_y[l_indices]
                m_X, m_y = c_X[m_indices], c_y[m_indices]
                r_X, r_y = c_X[r_indices], c_y[r_indices]
                if l_y.size > 0:
                    node_id += 1
                    kd_tree.add_edge(node, node_id)
                    kd_tree.nodes[node_id]["X"] = l_X
                    kd_tree.nodes[node_id]["y"] = l_y
                    kd_tree.nodes[node_id]["node_type"] = 0
                    new_nodes.append(node_id)
                
                if r_y.size > 0:
                    node_id += 1
                    kd_tree.add_edge(node, node_id)
                    kd_tree.nodes[node_id]["X"] = r_X
                    kd_tree.nodes[node_id]["y"] = r_y
                    kd_tree.nodes[node_id]["node_type"] = 1
                    new_nodes.append(node_id)

                if m_y.size != 0:
                    kd_tree.nodes[node]["points"] = (m_X, m_y)
                else:
                    kd_tree.nodes[node]["points"] = None
            else:
                kd_tree.nodes[node]['node_type'] = 'leaf'
                kd_tree.nodes[node]['points'] = (c_X, c_y)
            
        i += 1
        no_tag_nodes = new_nodes
        
    return kd_tree

In [60]:
data = pd.DataFrame({'x1': [1, 2, 2, 3], 'x2': [-2, -3, -4, 0], 'y': [0, 1, 0, 1]})

In [22]:
data.y.size

4

In [35]:
data[data.x1 == 2]

,x1,x2,y
1,2,-3,1
2,2,-4,0


In [70]:
kd_tree = generate_kd_tree(data)

In [66]:
x, y = kd_tree.nodes[2]["points"]

In [112]:
np.sort([2,1,2,2,3])

array([1, 2, 2, 2, 3])

**算法3.3 （基于kd树的k最优近邻搜索）**

- 输入: 已构造的kd树，目标点x，邻居数量k
- 输出: x的k个最近邻k_list
- 算法过程
    - 在kd树种找到包含目标点x的叶结点：从根节点出发，递归地向下访问kd树，若目标点x当前维的坐标小于切分点的坐标，则移动到左子结点，否则移动到右子结点。直到子结点为叶结点为止。
    - 将此叶结点添加至k_list
    - 递归地向上级结点搜索回退，在每个结点t_node进行以下操作：
        - 如果$len(k\_list) < k$, 则将t_node添加至k_list：如果该点到x的距离大于k_list中所有结点至x的距离，继续往上寻找

In [72]:
def search_kd_tree(x, node, kd_tree):
    '''
    搜索node在哪个区域
    '''
    if kd_tree.nodes[node]['node_type'] != 'leaf':
        dim = kd_tree.nodes[node]['dim']
        median = kd_tree.nodes[node]['median']
        succesors = sorted(kd_tree.successors(node), key=lambda x: kd_tree.nodes[x]["node_type"])
        if x[dim] == median:  # 点在内部结点上，同时搜索左右两个子结点
            return [search_kd_tree(x, succesors[0], kd_tree), search_kd_tree(x, succesors[1], kd_tree)]
        elif x[dim] < median:  # 左子节点
            return search_kd_tree(x, succesors[0], kd_tree)
        else:  # 右子结点
            return search_kd_tree(x, succesors[1], kd_tree)
    else:
        return node

In [ ]:
def find_k_neighbors(x, k, kd_tree):
    '''
    从叶结点回退
    k_list保存离x最近的k个点（注意，不是结点，一个结点可能保存多个数据点）
    '''
    k_list = []  # 保存k个最近邻居
    travel_list = []  # 结点遍历历史
    c_node = search_kd_tree(x, 0, kd_tree)  # 定位到的结点
    # 如果c_node为非叶结点，则应该继续搜索这个内部结点的所有子孙结点
    if kd_tree.nodes[c_node]["node_type"] != 'leaf':
        c_x = kd_tree.nodes[c_node]["X"].values
        travel_list.append(c_node)
        k_list.extend([(xi, distance(x, xi)) for xi in c_x])
        k_list = sorted(k_list, key=lambda x: x[1])[:k]  # 如果找到了超过k个最近点，则剔除距离最远的几个点
    else:
        
    
    while True:
        predeccessors = list(kd_tree.predeccessors(node))
        if predeccessors:
            p_node = predeccessors[0]
            X, y = kd_tree.nodes[p_node]["points"]
            dim = kd_tree.nodes[p_node]["dim"]
            xi = X[X.columns[dim]]
            dists = [(X.index[i], distance(xi, x)) for i, xi in enumerate(X.values)]
        else:
            
        
    

In [73]:
X = np.array([[1,2,3], [4, 5, 6]])
x = np.array([1,1,1])

In [88]:
for i in X.values:
    print(distance(i, x))

2.23606797749979
7.0710678118654755


In [97]:
for i, d in enumerate(data.values):
    print(i,d)

0 [ 1 -2  0]
1 [ 2 -3  1]
2 [ 2 -4  0]
3 [3 0 1]


In [105]:
data.x1.values

0    1
1    2
2    2
3    3
Name: x1, dtype: int64

In [96]:
data.index[0]

0

In [108]:
data.iloc[1:2].values

array([[ 2, -3,  1]])